## Accessing Mistral 7B

## Trainning Mistral 7B - Conversational with MetaMathQA_395k

### Bước 0: Cài thư viện cần thiết

In [1]:
%%capture
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install unsloth vllm
# !pip install --upgrade transformers==4.52.3

### Bước 0: Đăng nhập HuggingFace để lấy mô hình 

In [2]:
!huggingface-cli login --token $secret_hf

usage: huggingface-cli <command> [<args>] login [-h] [--token TOKEN] [--add-to-git-credential]
huggingface-cli <command> [<args>] login: error: argument --token: expected one argument


In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

In [4]:
from huggingface_hub import login
login(token=hf_token, new_session=False)

### Bước 1: Load mô hình Mistral 4bit từ Unsloth

In [5]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Chiều dài tối đa 1 đoạn văn bản
dtype = None           # Tự động chọn (float16, bfloat16) theo GPU
load_in_4bit = True    # Dùng mô hình lượng tử hóa 4-bit
lora_rank = 32

# Load model đã quant hóa 4bit + tự động chia lên GPU phù hợp
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-30 01:54:22.342840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751248462.730696      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751248462.846832      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-30 01:54:46 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-30 01:54:46 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.6.8: Fast Mistral patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/mistral-7b-instruct-v0.3-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 160.
Unsloth: vLLM's KV Cache can

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

...

INFO 06-30 01:57:08 [model_runner.py:1592] Graph capturing finished in 49 secs, took 0.53 GiB
INFO 06-30 01:57:08 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 62.20 seconds
Unsloth: Just some info: will skip parsing ['k_norm', 'q_norm', 'pre_feedforward_layernorm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['k_norm', 'q_norm', 'pre_feedforward_layernorm', 'post_feedforward_layernorm']


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

### Bước 2: Áp dụng LoRA

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


###  Bước 3: Gán Chat Template kiểu ChatML. Load và xử lý dữ liệu MetaMathQA

### Train/test split

In [8]:
from datasets import load_dataset

dataset = load_dataset("meta-math/MetaMathQA", split="train")
print(len(dataset))

README.md: 0.00B [00:00, ?B/s]

MetaMathQA-395K.json:   0%|          | 0.00/396M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/395000 [00:00<?, ? examples/s]

395000


In [9]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [10]:
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

### Format dạng hội thoại với system instruction

In [11]:
from unsloth.chat_templates import get_chat_template

# Gán template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",  # hoặc "mistral", "unsloth"
    map_eos_token=True,
)

# System Instruction
system_instruction_fixed = """Below is an instruction that describes a mathematical task.
Write a response that thoroughly solves the given problem.
Before solving, develop a clear step-by-step chain of reasoning to ensure accuracy and logical coherence.

### Instruction:
You are a mathematics expert with advanced knowledge in mathematical reasoning, problem-solving, and proof techniques. You think outloud and consider various aspects before giving any concrete answers."""

def formatting_prompts_func_conversational_structured(examples):
    queries = examples["query"]
    responses = examples["response"]
    texts = []

    for q, r in zip(queries, responses):
        messages = [
            {"role": "system", "content": system_instruction_fixed},
            {"role": "user", "content": q},
            {"role": "assistant", "content": r},
        ]
        formatted = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(formatted)

    return {"text": texts}


Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [12]:
train_dataset = train_dataset.map(formatting_prompts_func_conversational_structured, batched=True)
eval_dataset = eval_dataset.map(formatting_prompts_func_conversational_structured, batched=True)

Map:   0%|          | 0/316000 [00:00<?, ? examples/s]

Map:   0%|          | 0/79000 [00:00<?, ? examples/s]

### Tokenize train and test dataset

In [ ]:
# # Tokenize
# def tokenize_func(example):
#     tokenized = tokenizer(
#         example["text"],
#         truncation=True,
#         max_length=2048,
#         padding="max_length",
#     )
#     tokenized["labels"] = tokenized["input_ids"].copy()
#     return tokenized


In [ ]:
# tokenized_train = train_dataset.map(tokenize_func, batched=False)
# tokenized_eval = eval_dataset.map(tokenize_func, batched=False)

In [ ]:
# from datasets import load_dataset
# from unsloth.chat_templates import get_chat_template

# # 1. Gán template đúng kiểu Mistral Instruct
# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "mistral",  # CHUẨN CHO MISTRAL INSTRUCT V0.3
#     mapping = {"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
#     map_eos_token = True,
# )

# # 2. Tải và chọn phần dữ liệu từ MetaMathQA
# dataset = load_dataset("meta-math/MetaMathQA", split="train")
# dataset = dataset.select(range(100000, 200000))  # Lấy từ dòng 100001 đến 200000
# print("Số mẫu:", len(dataset))

# # 3. Chuyển đổi thành định dạng hội thoại ShareGPT-style
# def convert_to_conversation(example):
#     return {
#         "conversations": [
#             {"from": "human", "value": example["query"]},
#             {"from": "gpt", "value": example["response"]},
#         ]
#     }

# dataset = dataset.map(convert_to_conversation)

# # 4. Format theo chat_template đã gán
# def formatting_prompts_func(examples):
#     texts = [
#         tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
#         for convo in examples["conversations"]
#     ]
#     return {"text": texts}

# dataset = dataset.map(formatting_prompts_func, batched=True)

# # 5. Tokenize
# def tokenize_func(example):
#     tokenized = tokenizer(
#         example["text"],
#         truncation=True,
#         max_length=2048,
#         padding="max_length",
#     )
#     tokenized["labels"] = tokenized["input_ids"].copy()
#     return tokenized

# tokenized_dataset = dataset.map(tokenize_func, batched=False)

# # Kết quả: tokenized_dataset sẵn sàng để huấn luyện với SFTTrainer

### Bước 5: Huấn luyện bằng `SFTTrainer`

In [ ]:
# from trl import SFTConfig, SFTTrainer

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,             # Dataset chứa field "text"
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     dataset_num_proc = 2,
#     packing = False,                     # Để True nếu các mẫu ngắn
#     args = SFTConfig(
#         output_dir = "outputs",
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         max_steps = 60,                  # Hoặc dùng num_train_epochs=1
#         learning_rate = 1e-4,
#         optim = "adamw_8bit",            # Tối ưu cho 4bit
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         logging_steps = 1,
#         report_to = "none",
#         seed = 3407,
#     ),
# )

# trainer.train()


In [13]:
from trl import SFTConfig, SFTTrainer

# Khởi tạo Trainer để huấn luyện mô hình với các thiết lập đã tối ưu cho toán học
trainer = SFTTrainer(
    model = model,                          # Mô hình đã được chuẩn bị (có thể LoRA hoặc fine-tuned)
    tokenizer = tokenizer,                  # Tokenizer phù hợp với mô hình
    train_dataset = train_dataset,        # Dữ liệu đã được token hóa cho huấn luyện
    eval_dataset = eval_dataset,          # Dữ liệu để đánh giá trong quá trình train
    dataset_text_field = "text",            # Trường chứa chuỗi đầu vào
    max_seq_length = 2048,                  # Độ dài tối đa mỗi chuỗi, phù hợp với Mistral
    dataset_num_proc = 2,                   # Số tiến trình song song để xử lý dữ liệu
    packing = False,                        # Không gộp nhiều mẫu ngắn lại – tốt hơn cho toán học vì mẫu thường dài
    args = SFTConfig(
        output_dir = "outputs",             # Thư mục để lưu checkpoint và log
        per_device_train_batch_size = 2,    # Batch size mỗi GPU – tăng nếu đủ VRAM
        per_device_eval_batch_size = 2,     # Batch size khi đánh giá – không ảnh hưởng tốc độ train
        gradient_accumulation_steps = 4,    # Số bước tích lũy gradient – tăng nếu VRAM hạn chế
        warmup_steps = 60,                  # Số bước đầu warmup learning rate (5% ~ 1000 steps) => Số chính xác: 60
        max_steps = 1200,                   # Tổng số bước huấn luyện (bạn có thể giảm nếu cần) => Số chính xác: 1200
        learning_rate = 2e-4,               # Learning rate – thường dùng 2e-4 cho LoRA
        optim = "adamw_8bit",               # Optimizer nhẹ, phù hợp với mô hình 4-bit
        weight_decay = 0.01,                # Tránh overfitting nhẹ
        lr_scheduler_type = "cosine",       # Tăng giảm learning rate mượt mà hơn “linear”
        logging_steps = 10,                 # In log sau mỗi 10 bước
        report_to = "none",                 # Nếu muốn theo dõi bằng wandb thì đổi thành "wandb"
        seed = 3407,                        # Đảm bảo kết quả reproducible nếu cần
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/316000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/79000 [00:00<?, ? examples/s]

In [14]:
# Bắt đầu quá trình huấn luyện
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 316,000 | Num Epochs = 1 | Total steps = 1,200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080/7,000,000,000 (1.20% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.406100
20,0.679800
30,0.451500
40,0.410300
50,0.375200
60,0.366600
70,0.360500
80,0.374900
90,0.339500
100,0.373000


In [22]:
trainer_stats

TrainOutput(global_step=1200, training_loss=0.3191232430934906, metrics={'train_runtime': 25687.4702, 'train_samples_per_second': 0.747, 'train_steps_per_second': 0.047, 'total_flos': 2.17223099365589e+17, 'train_loss': 0.3191232430934906})

### Bước 7: Inference sau khi huấn luyện

In [23]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer
import torch

# Kích hoạt chế độ inference nhanh
FastLanguageModel.for_inference(model)

# Gắn lại chat template chuẩn chatml (hoặc mistral nếu fine-tune theo kiểu đó)
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    map_eos_token=True,
)

# Hàm hỏi đáp inference
def chat(question, max_new_tokens=512, stream=False):
    messages = [{"from": "human", "value": question}]
    
    # Tạo prompt theo chuẩn chat
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Tùy chọn stream ra màn hình trực tiếp
    if stream:
        streamer = TextStreamer(tokenizer)
        _ = model.generate(
            input_ids=inputs,
            streamer=streamer,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            top_k=1,
            repetition_penalty=1.1,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    else:
        # Sinh ra kết quả hoàn chỉnh
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            top_k=1,
            repetition_penalty=1.1,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id,
        )

        # Giải mã kết quả
        result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        # Cắt phần prompt nếu cần
        response = result.split(question.strip())[-1].strip()
        return response


In [25]:
response = chat("If $x - y = X and $x + y = 12$, what is the value of $x$? If we know the answer to the above question is 9, what is the value of unknown variable X?")
print(response) # Đáp án: 6 => Sai

<|im_start|>assistant
We want to find the value of $X$ in the given situation.
We are given that $x - y = X$ and $x + y = 12$.
To find the value of $x$, we can solve this system of equations.
We can start by solving the second equation for $y$:
$x + y = 12$
Subtracting $x$ from both sides of the equation, we get:
$y = 12 - x$
Now, substitute this value of $y$ into the first equation:
$x - (12 - x) = X$
Simplifying, we have:
$-x + 12 = X$
Adding $x$ to both sides of the equation, we find:
$12 = X + x$
Given that the value of $x$ is 9, we can substitute it into the equation:
$12 = X + 9$
Subtracting 9 from both sides of the equation, we get:
$3 = X$
The value of X is 3.
The answer is: 3


In [26]:
chat("A radio show plays for 3 hours a day. They split their show into talking segments, ad breaks and songs. Talking segments last 10 minutes each, ad breaks last 5 minutes each and songs are played throughout the rest of the show. If the radio show includes 3 talking segments and x ad breaks in today’s show, how long, in minutes, does the show play songs? If we know the answer to the above question is 125, what is the value of unknown variable x?", stream=True)
# Đúng, đáp = 5

<|im_start|>user
A radio show plays for 3 hours a day. They split their show into talking segments, ad breaks and songs. Talking segments last 10 minutes each, ad breaks last 5 minutes each and songs are played throughout the rest of the show. If the radio show includes 3 talking segments and x ad breaks in today’s show, how long, in minutes, does the show play songs? If we know the answer to the above question is 125, what is the value of unknown variable x?<|im_end|>
<|im_start|>assistant
The radio show plays for 3 hours, which is equal to 3 * 60 = 180 minutes.
Talking segments last 10 minutes each, so 3 talking segments will last 3 * 10 = 30 minutes.
Ad breaks last 5 minutes each, so x ad breaks will last x * 5 = 5x minutes.
The remaining time is spent playing songs, so the total time spent on songs is 180 - 30 - 5x = 150 - 5x minutes.
We are given that the total time spent on songs is 125 minutes, so we can write: 150 - 5x = 125.
Solving for x, we get: x = 5.
The value of x is 5.
#

In [27]:
response = chat("In a charity race to raise money for hurricane victims, thirty students participated. Ten of them raised $20 each, while the remaining students raised $30 each. What is the total amount of money raised by the students in the race?")
print(response)
# Đúng, đáp án 800

<|im_start|>assistant
Ten students raised $20 each, so they contributed 10 * $20 = $200.
The remaining twenty students raised $30 each, so they contributed 20 * $30 = $600.
Therefore, the total amount of money raised by the students is $200 + $600 = $800.
#### 800
The answer is: 800


In [29]:
chat("Which two-digit positive integer is one more than a multiple of 2, 3, 4, 5, and 6?", stream=True)
# Sai hoàn toàn, đáp là: 61

<|im_start|>user
Which two-digit positive integer is one more than a multiple of 2, 3, 4, 5, and 6?<|im_end|>
<|im_start|>assistant
The smallest two-digit positive integer that is one more than a multiple of 2, 3, 4, 5, and 6 is $10$.
Since $10$ is divisible by all the given numbers, it is the only possible answer.
The answer is: 10<|im_end|>


In [30]:
chat("Simplify: $|{-3^2+4}|$", stream=True)
# Sai, đáp: 55

<|im_start|>user
Simplify: $|{-3^2+4}|$<|im_end|>
<|im_start|>assistant
We can simplify the expression inside the absolute value first.
$-3^2 + 4 = -9 + 4 = -5$.
So, $|-5| = \boxed{5}$.The answer is: 5<|im_end|>


In [31]:
chat("Steven's teacher sends the class an assignment to collect x different fruit seeds. Apples average 6 seeds, pears average 2 seeds, and grapes average 3 seeds. Steven has set aside 4 apples, 3 pears, and 9 grapes to extract their seeds. How many more seeds does he need to fulfill his assignment? If we know the answer to the above question is 3, what is the value of unknown variable x?", stream=True)
# Đáp án 60 => Đúng

<|im_start|>user
Steven's teacher sends the class an assignment to collect x different fruit seeds. Apples average 6 seeds, pears average 2 seeds, and grapes average 3 seeds. Steven has set aside 4 apples, 3 pears, and 9 grapes to extract their seeds. How many more seeds does he need to fulfill his assignment? If we know the answer to the above question is 3, what is the value of unknown variable x?<|im_end|>
<|im_start|>assistant
The number of seeds in each type of fruit can be calculated as follows:
Number of seeds in apples = Number of apples * Average number of seeds per apple = 4 * 6 = 24
Number of seeds in pears = Number of pears * Average number of seeds per pear = 3 * 2 = 6
Number of seeds in grapes = Number of grapes * Average number of seeds per grape = 9 * 3 = 27
The total number of seeds Steven has set aside is the sum of the seeds in each type of fruit: 24 + 6 + 27 = 57
We are given that Steven needs to collect x different fruit seeds, so the number of additional seeds he 

In [32]:
chat(
"""I am a two-digit number.
The sum of my digits is 11.
If you reverse my digits, the new number = the original number + 27.
What number am I?
""",
    stream=True
)
# Đáp án: 47 => Sai

<|im_start|>user
I am a two-digit number.
The sum of my digits is 11.
If you reverse my digits, the new number = the original number + 27.
What number am I?
<|im_end|>
<|im_start|>assistant
Let's call the two-digit number $AB$.
We are given that $A+B=11$ and $AB+27=BA+27$.
Subtracting these equations, we get $(A-B)(B-A)=0$.
Since $A\neq B$, we must have $A=B$.
Therefore, $AB=AA=A^2$.
We know that $A^2+27=BA+27=11A$.
Substituting $A=B$, we get $A^2+27=11A$.
Rearranging, we have $A^2-11A+27=0$.
Factoring, we find $(A-3)(A-9)=0$.
Since $A$ is a two-digit number, we must have $A=3$.
Therefore, the two-digit number is $\boxed{33}$.
The answer is: 33<|im_end|>


In [33]:
chat("At Frank's Fruit Market, 3 bananas cost as much as 2 apples, and 6 apples cost as much as 4 oranges. How many oranges cost as much as 18 bananas?", stream=True)
# Sai, đáp án là 8

<|im_start|>user
At Frank's Fruit Market, 3 bananas cost as much as 2 apples, and 6 apples cost as much as 4 oranges. How many oranges cost as much as 18 bananas?<|im_end|>
<|im_start|>assistant
Let $b$ be the price of one banana, $a$ be the price of one apple, and $o$ be the price of one orange.
From the given information, we have the following equations:
\[3b = 2a\]
\[6a = 4o\]
We can solve these equations to find the values of $b$, $a$, and $o$.
First, we can solve the first equation for $a$:
\[3b = 2a \Rightarrow a = \frac{3}{2}b\]
Next, we can substitute this value of $a$ into the second equation:
\[\frac{3}{2}b = 4o \Rightarrow b = \frac{8}{3}o\]
Now, we can use either equation to find the price of one banana in terms of the price of one orange:
\[b = \frac{8}{3}o \Rightarrow o = \frac{3}{8}b\]
Finally, we can use this expression for $o$ to find the number of oranges that cost as much as 18 bananas:
\[\frac{3}{8}b \cdot 18 = 9b\]
Since $b$ is the price of one banana, we can rewri

In [34]:
chat("Jill bought 5 packs of red bouncy balls and 4 packs of yellow bouncy balls. Each package contained 18 bouncy balls. How many more red bouncy balls than yellow bouncy balls did Jill buy?", stream=True)
# Đúng, đáp án là 18

<|im_start|>user
Jill bought 5 packs of red bouncy balls and 4 packs of yellow bouncy balls. Each package contained 18 bouncy balls. How many more red bouncy balls than yellow bouncy balls did Jill buy?<|im_end|>
<|im_start|>assistant
Jill bought 5 packs x 18 = <<5*18=90>>90 red bouncy balls.
She also bought 4 packs x 18 = <<4*18=72>>72 yellow bouncy balls.
Therefore, she bought 90 - 72 = <<90-72=18>>18 more red bouncy balls than yellow bouncy balls.
#### 18
The answer is: 18<|im_end|>


In [35]:
chat("A 1$\%$ late charge was added to Jenna's bill on the $30^{\text{th}}$ day past its due date. The resulting total was then increased by 1$\%$ because she did not pay the bill in the next 30 days either. Her original bill was $\$400$. Exactly how much is the bill now?", stream=True)
# Sai, đáp án là 400

<|im_start|>user
A 1$\%$ late charge was added to Jenna's bill on the $30^{	ext{th}}$ day past its due date. The resulting total was then increased by 1$\%$ because she did not pay the bill in the next 30 days either. Her original bill was $\$400$. Exactly how much is the bill now?<|im_end|>
<|im_start|>assistant
The first late fee of 1% of the bill amount, which is $400 \times 1\% = \$4$, was added on the 30th day past the due date.
After this, the bill amount became $400 + 4 = \$404$.
On the next 30 days, another 1% late fee was added, which is $404 \times 1\% = \$4$.
So, the final bill amount is $404 + 4 = \boxed{\$408}$.
The answer is: 408<|im_end|>


In [37]:
chat("Compute $\dbinom{16}{5}$.", stream=True)
# Đúng, đáp số 4368

<|im_start|>user
Compute $\dbinom{16}{5}$.<|im_end|>
<|im_start|>assistant
We can use the formula for combinations to compute this. The number of ways to choose 5 items from a set of 16 is given by $\binom{16}{5} = \frac{16!}{5!(16-5)!} = \frac{16!}{5!11!} = \frac{16\times15\times14\times13\times12}{5\times4\times3\times2\times1} = \boxed{4368}$.The answer is: 4368<|im_end|>


In [ ]:
# from transformers import pipeline

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# questions = [
#     "What is the integral of x^2?",
#     "Define a group in abstract algebra.",
#     "Prove that the square root of 2 is irrational.",
# ]

# for q in questions:
#     prompt = tokenizer.apply_chat_template(
#         [{"role": "user", "content": q}],
#         tokenize=False,
#         add_generation_prompt=True,
#     )
#     output = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7)
#     print(f"\n❓ {q}\n🧠 {output[0]['generated_text']}\n" + "-"*60)


### Bước 8: Lưu mô hình

#### Tải mô hình

In [15]:
model.save_pretrained("mistral-metamathqa-lora-365k")
tokenizer.save_pretrained("mistral-metamathqa-lora-365k")

('mistral-metamathqa-lora-365k/tokenizer_config.json',
 'mistral-metamathqa-lora-365k/special_tokens_map.json',
 'mistral-metamathqa-lora-365k/tokenizer.json')

In [16]:
import shutil

shutil.make_archive(
    base_name="/kaggle/working/mistral-metamathqa-lora-365k",  # Tên file zip (không cần .zip ở đây)
    format="zip",  # Có thể là: 'zip', 'tar', 'gztar', 'bztar', 'xztar'
    root_dir="/kaggle/working/mistral-metamathqa-lora-365k"
)

'/kaggle/working/mistral-metamathqa-lora-365k.zip'

### Load mô hình lên HuggingFace

In [17]:
from huggingface_hub import create_repo

create_repo("mistral-metamathqa-lora-365k", private=False)  # hoặc private=False nếu muốn công khai

RepoUrl('https://huggingface.co/hahuy2004/mistral-metamathqa-lora-365k', endpoint='https://huggingface.co', repo_type='model', repo_id='hahuy2004/mistral-metamathqa-lora-365k')

In [18]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="/kaggle/working/mistral-metamathqa-lora-365k",
    repo_id="hahuy2004/mistral-metamathqa-lora-365k",  # sửa lại đúng username Hugging Face của bạn
    repo_type="model"
)


adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hahuy2004/mistral-metamathqa-lora-365k/commit/7866f26e8a811c440edbbe58f9c30f937cc424f8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='7866f26e8a811c440edbbe58f9c30f937cc424f8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hahuy2004/mistral-metamathqa-lora-365k', endpoint='https://huggingface.co', repo_type='model', repo_id='hahuy2004/mistral-metamathqa-lora-365k'), pr_revision=None, pr_num=None)